In [ ]:
# !pip uninstall tensorflow
# !pip install tensorflow

^C


In [ ]:
# !pip install tensorflow
# !pip install scikit-learn
# !pip install tensorflow==2.11
# !nvidia-smi
# !pip install numpy==1.24

Sat Nov  2 23:41:39 2024       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 565.90                 Driver Version: 565.90         CUDA Version: 12.7     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                  Driver-Model | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce RTX 4070 ...  WDDM  |   00000000:01:00.0 Off |                  N/A |
| N/A   48C    P3             11W /   57W |       0MiB /   8188MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [2]:
# import tensorflow as tf

# # TensorFlow에서 GPU 장치가 인식되는지 확인
# print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))
# print("Available devices:", tf.config.list_physical_devices())

Num GPUs Available:  0
Available devices: [PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU')]


In [ ]:
# import tensorflow as tf

# print("Is GPU available:", tf.test.is_gpu_available())
# print("Is cuDNN enabled:", tf.test.is_built_with_cuda())

Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.
Is GPU available: False
Is cuDNN enabled: False


In [ ]:
# !nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2022 NVIDIA Corporation
Built on Wed_Sep_21_10:41:10_Pacific_Daylight_Time_2022
Cuda compilation tools, release 11.8, V11.8.89
Build cuda_11.8.r11.8/compiler.31833905_0


In [ ]:
# import tensorflow as tf
# print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

Num GPUs Available:  0


In [5]:
from PIL import Image
import glob

def resize_images(img_path):
	images=glob.glob(img_path + "/*.jpg")  
    
	print(len(images), " images to be resized.")

    # 파일마다 모두 224*224 사이즈로 바꾸어 저장합니다.
	target_size=(224,224)
	for img in images:
		old_img=Image.open(img)
		new_img=old_img.resize(target_size,Image.ANTIALIAS)
		new_img.save(img, "JPEG")
    
	print(len(images), " images resized.")

In [6]:
# 가위 이미지가 저장된 디렉토리 아래의 모든 jpg 파일을 읽어들여서
image_dir_path = r"moving\rockgame\dataset\scissors"
resize_images(image_dir_path)

print("가위 이미지 resize 완료!")

0  images to be resized.
0  images resized.
가위 이미지 resize 완료!


In [7]:
# 바위 이미지가 저장된 디렉토리 아래의 모든 jpg 파일을 읽어들여서
image_dir_path = r"moving\rockgame\dataset\rock"
resize_images(image_dir_path)

print("바위 이미지 resize 완료!")

0  images to be resized.
0  images resized.
바위 이미지 resize 완료!


In [8]:
# 보 이미지가 저장된 디렉토리 아래의 모든 jpg 파일을 읽어들여서
image_dir_path = r"moving\rockgame\dataset\paper"
resize_images(image_dir_path)

print("보 이미지 resize 완료!")

0  images to be resized.
0  images resized.
보 이미지 resize 완료!


In [9]:
import numpy as np
import os
import matplotlib.pyplot as plt
from PIL import Image
from sklearn.model_selection import train_test_split

def load_data(img_path, number_of_data=3914):  # 가위바위보 이미지 개수 총합에 주의하세요.
    # 가위 : 0, 바위 : 1, 보 : 2
    img_size=224
    color=3
    #이미지 데이터와 라벨(가위 : 0, 바위 : 1, 보 : 2) 데이터를 담을 행렬(matrix) 영역을 생성합니다.
    imgs=np.zeros(number_of_data*img_size*img_size*color,dtype=np.int32).reshape(number_of_data,img_size,img_size,color)
    labels=np.zeros(number_of_data,dtype=np.int32)

    idx=0
    for file in glob.iglob(img_path+'/scissors/*.jpg'):
        img = Image.open(file).resize((224, 224))  # 크기를 (28, 28)로 변경
        img = np.array(img, dtype=np.int32)
        imgs[idx,:,:,:]=img    # 데이터 영역에 이미지 행렬을 복사
        labels[idx]=0   # 가위 : 0
        idx=idx+1

    for file in glob.iglob(img_path+'/rock/*.jpg'):
        img = Image.open(file).resize((224, 224))  # 크기를 (28, 28)로 변경
        img = np.array(img, dtype=np.int32)
        imgs[idx,:,:,:]=img    # 데이터 영역에 이미지 행렬을 복사
        labels[idx]=1   # 바위 : 1
        idx=idx+1  
    
    for file in glob.iglob(img_path+'/paper/*.jpg'):
        img = Image.open(file).resize((224, 224))  # 크기를 (28, 28)로 변경
        img = np.array(img, dtype=np.int32)
        imgs[idx,:,:,:]=img    # 데이터 영역에 이미지 행렬을 복사
        labels[idx]=2   # 보 : 2
        idx=idx+1
        
    print("학습데이터(x_train)의 이미지 개수는", idx,"입니다.")
    return imgs, labels

image_dir_path = "dataset" #폴더명
(X, y)=load_data(image_dir_path)
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.3, random_state=0, stratify=y)
x_train_norm = X_train/255.0   # 입력은 0~1 사이의 값으로 정규화
x_val_norm = X_val/255.0
print("x_train shape: {}".format(X_train.shape))
print("y_train shape: {}".format(y_train.shape))

학습데이터(x_train)의 이미지 개수는 3914 입니다.
x_train shape: (2739, 224, 224, 3)
y_train shape: (2739,)


In [10]:
# for i in range(100):
#     plt.imshow(x_train[i])
#     print('라벨 : ', y_train[i])
#     plt.show()

In [11]:
import tensorflow as tf
from tensorflow import keras
import numpy as np

model=keras.models.Sequential()
model.add(keras.layers.Conv2D(32, (3,3), activation='relu', input_shape=(224,224,3)))
model.add(keras.layers.MaxPool2D(2,2))
model.add(keras.layers.Conv2D(64, (3,3), activation='relu'))
model.add(keras.layers.BatchNormalization())
model.add(keras.layers.MaxPooling2D((2,2)))
model.add(keras.layers.Dropout(0.5))
model.add(keras.layers.Flatten())
model.add(keras.layers.Dense(128, activation='relu', kernel_regularizer=keras.regularizers.l2(0.01)))
model.add(keras.layers.Dropout(0.5))
model.add(keras.layers.Dense(3, activation='softmax'))

# model = keras.models.Sequential()
# model.add(keras.layers.Conv2D(64, (5,5), activation='relu', input_shape=(224,224,3)))  # 필터와 커널 크기 변경
# model.add(keras.layers.MaxPool2D(3,3))  # 풀링 크기 변경
# model.add(keras.layers.Conv2D(128, (3,3), activation='relu'))
# model.add(keras.layers.MaxPooling2D((2,2)))
# model.add(keras.layers.Flatten())
# model.add(keras.layers.Dense(256, activation='relu'))  # Dense 레이어 유닛 증가
# model.add(keras.layers.Dense(20, activation='softmax'))


model.summary()

c:\Users\zqrc0\anaconda3\envs\motionpy310\lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 222, 222, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 111, 111, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 109, 109, 64)   │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 109, 109, 64)   │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 54, 54, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 54, 54, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 186624)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │    23,888,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 3)              │           387 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 23,908,035 (91.20 MB)

 Trainable params: 23,907,907 (91.20 MB)

 Non-trainable params: 128 (512.00 B)

In [16]:
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.preprocessing.image import ImageDataGenerator

optimizer = keras.optimizers.Adam(learning_rate=0.0001)

model.compile(optimizer=optimizer,
             loss='sparse_categorical_crossentropy',
             metrics=['accuracy'])

# ModelCheckpoint 콜백 설정
checkpoint = ModelCheckpoint(
    'best_model.keras',  # 경로를 지정하여 저장
    monitor='val_loss',
    save_best_only=True,
    mode='min'
)

# 데이터 증강
datagen = ImageDataGenerator(
    rotation_range=20,     # 이미지 회전
    width_shift_range=0.2, # 가로로 이동
    height_shift_range=0.2,# 세로로 이동
    shear_range=0.2,       # 시야각 변환
    zoom_range=0.2,        # 확대/축소
    horizontal_flip=True,  # 좌우 반전
    fill_mode='nearest'
)

# 증강된 이미지로 모델 학습
model.fit(datagen.flow(x_train_norm, y_train, batch_size=32),
          validation_data=(x_val_norm, y_val),
          steps_per_epoch=len(x_train_norm) // 32,
          epochs=10,
          callbacks=[checkpoint])   # checkpoint 콜백 추가

Epoch 1/10
85/85 ━━━━━━━━━━━━━━━━━━━━ 36s 408ms/step - accuracy: 0.6641 - loss: 1.0465 - val_accuracy: 0.5685 - val_loss: 1.3703
Epoch 2/10
85/85 ━━━━━━━━━━━━━━━━━━━━ 2s 26ms/step - accuracy: 0.6562 - loss: 1.0498 - val_accuracy: 0.5549 - val_loss: 1.4472
Epoch 3/10
85/85 ━━━━━━━━━━━━━━━━━━━━ 34s 400ms/step - accuracy: 0.6898 - loss: 1.0064 - val_accuracy: 0.6306 - val_loss: 0.9905
Epoch 4/10
85/85 ━━━━━━━━━━━━━━━━━━━━ 3s 32ms/step - accuracy: 0.6250 - loss: 1.0757 - val_accuracy: 0.6740 - val_loss: 0.9331
Epoch 5/10
85/85 ━━━━━━━━━━━━━━━━━━━━ 34s 397ms/step - accuracy: 0.6810 - loss: 1.0058 - val_accuracy: 0.7243 - val_loss: 0.9565
Epoch 6/10
85/85 ━━━━━━━━━━━━━━━━━━━━ 3s 32ms/step - accuracy: 0.8125 - loss: 0.8932 - val_accuracy: 0.7855 - val_loss: 0.8810
Epoch 7/10
85/85 ━━━━━━━━━━━━━━━━━━━━ 35s 406ms/step - accuracy: 0.7324 - loss: 0.9845 - val_accuracy: 0.7779 - val_loss: 0.8561
Epoch 8/10
85/85 ━━━━━━━━━━━━━━━━━━━━ 2s 26ms/step - accuracy: 0.6562 - loss: 0.9798 - val_accuracy: 0.

In [17]:
image_dir_path = "dataset/test"
(x_test, y_test)=load_data(image_dir_path,328)
x_test_norm = x_test/255.0

print("x_test shape : {}".format(x_test.shape))
print("y_test shape : {}".format(y_test.shape))

학습데이터(x_train)의 이미지 개수는 328 입니다.
x_test shape : (328, 224, 224, 3)
y_test shape : (328,)


In [18]:
# 저장된 최적의 모델 로드
best_model = keras.models.load_model('best_model.keras')

# 테스트 데이터로 평가
test_loss, test_accuracy = best_model.evaluate(x_test_norm, y_test, verbose=2)
print("test_loss : {}".format(test_loss))
print("test_accuracy : {}".format(test_accuracy))

11/11 - 1s - 68ms/step - accuracy: 0.5213 - loss: 1.4107
test_loss : 1.4106818437576294
test_accuracy : 0.5213414430618286


In [19]:
predicted_result = model.predict(x_test_norm)	# model이 추론한 확률값
predicted_labels = np.argmax(predicted_result, axis=1)

idx=100		# 값을 변경해서 찾아보자
print('model.predict() 결과 : ', predicted_result[idx])
print('model이 추론한 가장 가능성이 높은 결과 : ', predicted_labels[idx])
print('실제 데이터의 라벨 : ', y_test[idx])

11/11 ━━━━━━━━━━━━━━━━━━━━ 1s 51ms/step
model.predict() 결과 :  [0.61214197 0.3049278  0.08293022]
model이 추론한 가장 가능성이 높은 결과 :  0
실제 데이터의 라벨 :  0


In [20]:
# plt.imshow(x_test[idx], cmap=plt.cm.binary)
# plt.show()

In [21]:
import random
wrong_predict_list=[]
for i, _ in enumerate(predicted_labels):
    if predicted_labels[i] != y_test[i]:
        wrong_predict_list.append(i)
        
samples = random.choices(population=wrong_predict_list, k=10) # k값을 수정하면 더 많은 값의 수를 볼 수 있다.

for n in samples:
    print("예측확률분포: " + str(predicted_result[n]))
    print("라벨: " + str(y_test[n]) + ", 예측결과: " + str(predicted_labels[n]))
    # plt.imshow(x_test[n], cmap=plt.cm.binary)
    print(n)
    # plt.show()

예측확률분포: [0.3489523  0.25147802 0.39956966]
라벨: 1, 예측결과: 2
188
예측확률분포: [0.16961493 0.5169908  0.31339425]
라벨: 2, 예측결과: 1
215
예측확률분포: [0.3794391  0.42800057 0.19256034]
라벨: 2, 예측결과: 1
239
예측확률분포: [0.5556835  0.37876257 0.06555391]
라벨: 1, 예측결과: 0
135
예측확률분포: [0.45260236 0.36610842 0.18128923]
라벨: 2, 예측결과: 0
245
예측확률분포: [0.15900049 0.455881   0.38511854]
라벨: 2, 예측결과: 1
270
예측확률분포: [0.29302338 0.6007814  0.10619522]
라벨: 2, 예측결과: 1
278
예측확률분포: [0.5759145  0.2713161  0.15276934]
라벨: 1, 예측결과: 0
207
예측확률분포: [0.16215464 0.6120536  0.2257918 ]
라벨: 2, 예측결과: 1
319
예측확률분포: [0.41088057 0.408286   0.18083349]
라벨: 2, 예측결과: 0
218


In [22]:
model.save('dataset/model/model.keras', include_optimizer=False)